In [29]:
# PROCESS NEW SHORT TRAJECTORIES (100 ns conventional; 500 ns aMD; 50 ns conventional MD)
# Some trajectories have PerM and FtsB in different unit cells so need to rewrap; was not needed for initial submission

import MDAnalysis as mda
from MDAnalysis import transformations as trans

directories to analyze
dirs = ['/home/zach/Documents/md/ftsb-perm-zn-aMD/aMD defaults/ftsb-perm/openmm_amd_dual/third_run/',
       '/home/zach/Documents/md/ftsb-perm-zn-aMD/aMD defaults/ftsb-perm/openmm_amd_dual/fourth_run/',
       '/home/zach/Documents/md/ftsb-perm-zn-aMD/aMD defaults/ftsb-perm/openmm_amd_dual/fifth_run/',
       '/home/zach/Documents/md/ftsb-perm-zn-aMD/aMD defaults/ftsb-perm/openmm_amd_dual/sixth_run/']

# Can include MEMB segment in exported PDBs or opt not to do this; only protein-only PDBs are analyzed for new data
include_membrane = False
wrapped = True # Unwrap protein and recenter on protein; quite slow

# FTSB and PERM segments were accidentally given inverted names in CHARMM-GUI
swap_ftsb_perm = [True, True, True, True]

stride = 1 # interval between frames saved; change to 1 later

step1 = 20 # number of dcd in prerun
step2 = 100 # number of dcd in aMD
step3 = 10 # number of dcd in postrun # maybe able to increase to 10 before submission (50 ns each)

for i in range(len(dirs)):
    
    print('processing',i+1,'of',len(dirs),'data directories')
    f = dirs[i]
    
    # load initial conformation pdb and find protein atoms
    fNames= []
    
    # fNames = [f+'step5_input.pdb'] # lacks unit cell; do not include and only use output coordinates after some time
    
    for j in range(1, step1+1):
        fNames.append(f+'step6.7_aMD_prerun_'+str(j)+'.dcd')
    for j in range(1, step2+1):
        fNames.append(f+'step7_'+str(j)+'.dcd')
    for j in range(1, step3+1):
        fNames.append(f+'step8_aMD_postrun_'+str(j)+'.dcd')

    universe = mda.Universe(f+'step5_input.psf',fNames,in_memory=False)

    if swap_ftsb_perm[i]:
        ftsb = universe.select_atoms("segid PERM")
        ftsb.segments.segids = 'temp'
        perm = universe.select_atoms("segid FTSB")
        perm.segments.segids = 'PERM'
        ftsb = universe.select_atoms("segid temp")
        ftsb.segments.segids = 'FTSB'

    protein = universe.select_atoms('protein')

    if include_membrane:
        membrane = universe.select_atoms('segid MEMB')
        to_write = universe.select_atoms('protein or segid MEMB')
    else:
        to_write = protein
    
    if wrapped: # does not work now because there is no box information read from DCD even though it is there...
        print('wrapping')

    
        if include_membrane:
            workflow = [trans.unwrap(protein),
                    trans.center_in_box(protein, wrap=False),
                    trans.wrap(membrane,compound='fragments')]
        else:
            workflow = [trans.unwrap(protein),
                    trans.center_in_box(protein, wrap=False)]
        
        universe.trajectory.add_transformations(*workflow)

    to_write.write(f+'protein-only-initial-frame.pdb',frames=universe.trajectory[[0]])
    to_write.write(f+'protein-only.dcd',frames=universe.trajectory[1::stride])

processing 1 of 1 data directories
wrapping
